In [1]:
!pip install pydicom

import os
import pydicom
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import ipywidgets as widgets
import nibabel as nib
import numpy as np
import pyarrow.parquet as pq

from IPython.display import HTML
from tqdm import tqdm

from google.colab import drive
import os

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 19.8 MB/s eta 0:00:00


In [ ]:

def mount_and_navigate_to_dir(directory_path="/content/drive/MyDrive/"):
    """
    Mount Google Drive and navigate to a specific directory.

    Args:
    - directory_path (str): The directory to navigate to after mounting.
    """
    ROOT = "/content/drive"
    drive.mount(ROOT, force_remount=True)
    os.chdir(directory_path)
    print(f"Current directory: {os.getcwd()}")

mount_and_navigate_to_dir()

Mounted at /content/drive
Current directory: /content/drive/MyDrive


In [ ]:
import time

In [ ]:
# Original -> save as PNG


def load_resize_save_dcm_images(directory, target_size):
    # 디렉토리에서 DCM 파일 목록을 가져옴
    dcm_files = [f for f in os.listdir(directory)]
    dcm_files = sorted(dcm_files, key=lambda x: int(x.split('.')[0]))

    # 중간 3개의 파일 디렉토리만 뽑아서 DCM 파일 읽어옴
    image_path1 = os.path.join(directory, dcm_files[len(dcm_files)//2-1])
    image_path2 = os.path.join(directory, dcm_files[len(dcm_files)//2])
    image_path3 = os.path.join(directory, dcm_files[len(dcm_files)//2+1])

    image1 = pydicom.dcmread(image_path1)
    image2 = pydicom.dcmread(image_path2)
    image3 = pydicom.dcmread(image_path3)

    # 원하는 크기로 resize
    resized_image1 = cv2.resize(image1.pixel_array, (target_size[0], target_size[1]), interpolation=cv2.INTER_AREA).astype(np.uint8)
    resized_image2 = cv2.resize(image2.pixel_array, (target_size[0], target_size[1]), interpolation=cv2.INTER_AREA).astype(np.uint8)
    resized_image3 = cv2.resize(image3.pixel_array, (target_size[0], target_size[1]), interpolation=cv2.INTER_AREA).astype(np.uint8)

    # 이미지 합치기
    images = np.zeros((target_size[0], target_size[1], 3), dtype=np.uint8)
    images[..., 0] = resized_image1
    images[..., 1] = resized_image2
    images[..., 2] = resized_image3

    return images
start = time.time()

TRAIN_CSV = "rsna_data/train_series_meta.csv"
data = pd.read_csv(TRAIN_CSV)

for index, row in data.iloc[200:300].iterrows():
  # DCM 파일이 있는 디렉토리 경로 설정
  dcm_directory = 'rsna_data/train_images/' + str(int(row['patient_id'])) + '/' + str(int(row['series_id']))
  print(str(int(row['series_id'])))

  # DCM 파일을 로드하여 target size 크기의 배열로 가져옴
  target_size = (256,256)
  loaded_images = load_resize_save_dcm_images(dcm_directory, target_size)

  output_directory = 'png_test_lyd/'  # Define the directory where you want to save PNG files

  for i in range(loaded_images.shape[2]):
    # Construct a filename for each slice
    output_subdirectory = os.path.join(output_directory, str(int(row['patient_id'])), str(int(row['series_id'])), 'img_256x256_d1_original')
    os.makedirs(output_subdirectory, exist_ok=True)  # Create subdirectories if they don't exist

    filename = os.path.join(output_subdirectory, f"image_{i+1:03d}.png")

    # Save the slice as a PNG file
    cv2.imwrite(filename, loaded_images[:, :, i])
end = time.time()

spend = end - start

In [ ]:
spend

511.6110978126526

In [ ]:
import os
import cv2
import pydicom
import numpy as np
import pandas as pd
from multiprocessing import Pool, cpu_count

def load_resize_save_dcm_images(directory, target_size, patient_id, series_id, output_directory):
    dcm_files = [f for f in os.listdir(directory)]
    dcm_files = sorted(dcm_files, key=lambda x: int(x.split('.')[0]))

    image_path1 = os.path.join(directory, dcm_files[len(dcm_files)//2-1])
    image_path2 = os.path.join(directory, dcm_files[len(dcm_files)//2])
    image_path3 = os.path.join(directory, dcm_files[len(dcm_files)//2+1])

    image1 = pydicom.dcmread(image_path1)
    image2 = pydicom.dcmread(image_path2)
    image3 = pydicom.dcmread(image_path3)

    resized_image1 = cv2.resize(image1.pixel_array, target_size, interpolation=cv2.INTER_AREA).astype(np.uint8)
    resized_image2 = cv2.resize(image2.pixel_array, target_size, interpolation=cv2.INTER_AREA).astype(np.uint8)
    resized_image3 = cv2.resize(image3.pixel_array, target_size, interpolation=cv2.INTER_AREA).astype(np.uint8)

    images = np.zeros((target_size[0], target_size[1], 3), dtype=np.uint8)
    images[..., 0] = resized_image1
    images[..., 1] = resized_image2
    images[..., 2] = resized_image3

    for i in range(loaded_images.shape[2]):
        output_subdirectory = os.path.join(output_directory, str(patient_id), str(series_id), 'img_256x256_d1_original')
        os.makedirs(output_subdirectory, exist_ok=True)
        filename = os.path.join(output_subdirectory, f"image_{i+1:03d}.png")
        cv2.imwrite(filename, images[:, :, i])

def process_row(args):
    row, output_directory = args
    dcm_directory = os.path.join('rsna_data', 'train_images', str(int(row['patient_id'])), str(int(row['series_id'])))
    target_size = (256, 256)
    load_resize_save_dcm_images(dcm_directory, target_size, int(row['patient_id']), int(row['series_id']), output_directory)

start = time.time()
TRAIN_CSV = "rsna_data/train_series_meta.csv"
data = pd.read_csv(TRAIN_CSV)

output_directory = 'png_test_lyd/'
with Pool(cpu_count()) as p:
    p.map(process_row, [(row, output_directory) for _, row in data.iloc[100:200].iterrows()])
end = time.time()

spend = end - start

In [ ]:
spend

277.6847038269043